# Scheduling a Doubles Pickleball Tournament

My friend Steve asked for help in creating a schedule for a round-robin doubles pickleball tournament with 8 or 9 players on 2 courts. ([Pickleball](https://en.wikipedia.org/wiki/Pickleball) is a paddle/ball/net game played on a court that is smaller than tennis but larger than ping-pong.) 

To generalize: given *P* players and *C* available courts, we would like to create a **schedule**: a table where each row is a time period (a round of play), each column is a court, and each cell contains a game, which consists of two players partnered together and pitted against two other players. The preferences for the schedule are:

- Each player should partner with each other player exactly once (or as close to that as possible).
- Fewer rounds are better (in other words, try to fill all the courts each round).
- Each player should play against each other player twice, or as close to that as possible.
- A player should not be scheduled to play two games at the same time.

For example, here's a perfect schedule for *P*=8 players on *C*=2 courts:

    [([[1, 6], [2, 4]], [[3, 5], [7, 0]]),
     ([[1, 5], [3, 6]], [[2, 0], [4, 7]]),
     ([[2, 3], [6, 0]], [[4, 5], [1, 7]]),
     ([[4, 6], [3, 7]], [[1, 2], [5, 0]]),
     ([[1, 0], [6, 7]], [[3, 4], [2, 5]]),
     ([[2, 6], [5, 7]], [[1, 4], [3, 0]]),
     ([[2, 7], [1, 3]], [[4, 0], [5, 6]])]
     
This means that in the first round, players 1 and 6 partner against 2 and 4 on one court, while 3 and 5 partner against 7 and 0 on the other. There are 7 rounds.

My strategy for finding a good schedule is to use **hillclimbing**: start with an initial schedule, then repeatedly alter the schedule by swapping partners in one game with partners in another. If the altered schedule is better, keep it; if not, discard it. Repeat. 

## Coding it up

The strategy in more detail:

- First form all pairs of players, using `all_pairs(P)`.
- Put pairs together to form a list of games using `initial_games`.
- Use `Schedule` to create a schedule; it calls `one_round` to create each round and `scorer` to evaluate the schedule.
- Use `hillclimb` to improve the initial schedule: call `alter` to randomly alter a schedule, `Schedule` to re-allocate the games to rounds and courts, and `scorer` to check if the altered schedule's score is better.



(Note: with *P* players there are *P &times; (P - 1) / 2* pairs of partners; this is an even number when either *P* or *P - 1* is divisible by 4, so everything works out when, say, *P*=4 or *P*=9,  but for, say, *P*=10 there are 45 pairs, and so `initial_games` chooses to create 22 games, meaning that one pair of players never play together, and thus play one fewer game than everyone else.)

In [1]:
import random
from itertools   import combinations
from collections import Counter

#### Types

Player = int   # A player is an int: `1`
Pair   = list  # A pair is a list of two players who are partners: `[1, 2]`
Game   = list  # A game is a list of two pairs: `[[1, 2], [3, 4]]`
Round  = tuple # A round is a tuple of games: `([[1, 2], [3, 4]], [[5, 6], [7, 8]])`

class Schedule(list):
    """A Schedule is a list of rounds (augmented with a score and court count)."""
    def __init__(self, games, courts=2):
        games = list(games)
        while games: # Allocate games to courts, one round at a time
            self.append(one_round(games, courts))
        self.score = scorer(self)
        self.courts = courts
        
#### Functions
        
def hillclimb(P, C=2, N=100000):
    "Schedule games for P players on C courts by randomly altering schedule N times."
    sched = Schedule(initial_games(all_pairs(P)), C)
    for _ in range(N):
        sched = max(alter(sched), sched, key=lambda s: s.score)
    return sched

def all_pairs(P): return list(combinations(range(P), 2))

def initial_games(pairs):
    """An initial list of games: [[[1, 2], [3, 4]], ...].
    We try to have every pair play every other pair once, and
    have each game have 4 different players, but that isn't always true."""
    random.shuffle(pairs)
    games = []
    while len(pairs) >= 2:
        A = pairs.pop()
        B = first(pair for pair in pairs if disjoint(pair, A)) or pairs[0]
        games.append([A, B])
        pairs.remove(B)
    return games

def disjoint(A, B): 
    "Do A and B have disjoint players in them?"
    return not (players(A) & players(B))

def one_round(games, courts):
    """Place up to `courts` games into `round`, all with disjoint players."""
    round = []
    while True:
        G = first(g for g in games if disjoint(round, g))
        if not G or not games or len(round) == courts:
            return Round(round)
        round.append(G)
        games.remove(G)

def players(x): 
    "All distinct players in a pair, game, or sequence of games."
    return {x} if isinstance(x, Player) else set().union(*map(players, x))

def first(items): return next(items, None)

def pairing(p1, p2): return tuple(sorted([p1,  p2]))
        
def scorer(sched):
    "Score has penalties for a non-perfect schedule."
    penalty  =   50 * len(sched) # More rounds are worse (avoid empty courts)
    penalty += 1000 * sum(len(players(game)) != 4 # A game should have 4 players!
                          for round in sched for game in round)
    penalty +=    1 * sum(abs(c - 2) ** 3 + 8 * (c == 0) # Try to play everyone twice
                          for c in opponents(sched).values())
    return -penalty
    
def opponents(sched):
    "A Counter of {(player, opponent): times_played}."
    return Counter(pairing(p1, p2) 
                   for round in sched for A, B in round for p1 in A for p2 in B)
    
def alter(sched):
    "Modify a schedule by swapping two pairs."
    games = [Game(game) for round in sched for game in round] 
    G = len(games)
    i, j = random.sample(range(G), 2) # index into games
    a, b = random.choice((0, 1)), random.choice((0, 1)) # index into each game
    games[i][a], games[j][b] = games[j][b], games[i][a]
    return Schedule(games, sched.courts)

def report(sched):
    "Print information about this schedule."
    for i, round in enumerate(sched, 1):
        print('Round {}: {}'.format(i, '; '.join('{} vs {}'.format(*g) for g in round)))
    games = sum(sched, ())
    P = len(players(sched))
    print('\n{} games in {} rounds for {} players'.format(len(games), len(sched), P))
    opp = opponents(sched)
    fmt = ('{:2X}|' + P * ' {}' + '   {}').format
    print('Number of times each player plays against each opponent:\n')
    print('  |', *map('{:X}'.format, range(P)), ' Total')
    print('--+' + '--' * P + '  -----')
    for row in range(P):
        counts = [opp[pairing(row, col)] for col in range(P)]
        print(fmt(row, *[c or '-' for c in counts], sum(counts) // 2))

# 8 Player Tournament

I achieved (in a previous run) a perfect schedule for 8 players: the 14 games fit into 7 rounds, each player partners with each other once, and plays each individual opponent twice:

In [2]:
report([
 ([[1, 6], [2, 4]], [[3, 5], [7, 0]]),
 ([[1, 5], [3, 6]], [[2, 0], [4, 7]]),
 ([[2, 3], [6, 0]], [[4, 5], [1, 7]]),
 ([[4, 6], [3, 7]], [[1, 2], [5, 0]]),
 ([[1, 0], [6, 7]], [[3, 4], [2, 5]]),
 ([[2, 6], [5, 7]], [[1, 4], [3, 0]]),
 ([[2, 7], [1, 3]], [[4, 0], [5, 6]]) ])

Round 1: [1, 6] vs [2, 4]; [3, 5] vs [7, 0]
Round 2: [1, 5] vs [3, 6]; [2, 0] vs [4, 7]
Round 3: [2, 3] vs [6, 0]; [4, 5] vs [1, 7]
Round 4: [4, 6] vs [3, 7]; [1, 2] vs [5, 0]
Round 5: [1, 0] vs [6, 7]; [3, 4] vs [2, 5]
Round 6: [2, 6] vs [5, 7]; [1, 4] vs [3, 0]
Round 7: [2, 7] vs [1, 3]; [4, 0] vs [5, 6]

14 games in 7 rounds for 8 players
Number of times each player plays against each opponent:

  | 0 1 2 3 4 5 6 7  Total
--+----------------  -----
 0| - 2 2 2 2 2 2 2   7
 1| 2 - 2 2 2 2 2 2   7
 2| 2 2 - 2 2 2 2 2   7
 3| 2 2 2 - 2 2 2 2   7
 4| 2 2 2 2 - 2 2 2   7
 5| 2 2 2 2 2 - 2 2   7
 6| 2 2 2 2 2 2 - 2   7
 7| 2 2 2 2 2 2 2 -   7


# 9 Player Tournament

For 9 players, I can fit the 18 games into 9 rounds, but some players play each other 1 or 3 times:

In [3]:
report([
 ([[1, 7], [4, 0]], [[3, 5], [2, 6]]),
 ([[2, 7], [1, 3]], [[4, 8], [6, 0]]),
 ([[5, 0], [1, 6]], [[7, 8], [3, 4]]),
 ([[7, 0], [5, 8]], [[1, 2], [4, 6]]),
 ([[3, 8], [1, 5]], [[2, 0], [6, 7]]),
 ([[1, 4], [2, 5]], [[3, 6], [8, 0]]),
 ([[5, 6], [4, 7]], [[1, 8], [2, 3]]),
 ([[1, 0], [3, 7]], [[2, 8], [4, 5]]),
 ([[3, 0], [2, 4]], [[6, 8], [5, 7]]) ])

Round 1: [1, 7] vs [4, 0]; [3, 5] vs [2, 6]
Round 2: [2, 7] vs [1, 3]; [4, 8] vs [6, 0]
Round 3: [5, 0] vs [1, 6]; [7, 8] vs [3, 4]
Round 4: [7, 0] vs [5, 8]; [1, 2] vs [4, 6]
Round 5: [3, 8] vs [1, 5]; [2, 0] vs [6, 7]
Round 6: [1, 4] vs [2, 5]; [3, 6] vs [8, 0]
Round 7: [5, 6] vs [4, 7]; [1, 8] vs [2, 3]
Round 8: [1, 0] vs [3, 7]; [2, 8] vs [4, 5]
Round 9: [3, 0] vs [2, 4]; [6, 8] vs [5, 7]

18 games in 9 rounds for 9 players
Number of times each player plays against each opponent:

  | 0 1 2 3 4 5 6 7 8  Total
--+------------------  -----
 0| - 2 1 2 2 1 3 3 2   8
 1| 2 - 3 3 2 2 1 2 1   8
 2| 1 3 - 3 3 2 2 1 1   8
 3| 2 3 3 - 1 1 1 2 3   8
 4| 2 2 3 1 - 2 2 2 2   8
 5| 1 2 2 1 2 - 3 2 3   8
 6| 3 1 2 1 2 3 - 2 2   8
 7| 3 2 1 2 2 2 2 - 2   8
 8| 2 1 1 3 2 3 2 2 -   8


# 10 Player Tournament

With *P*=10 there is an odd number of pairings (45), so two players necessarily play one game less than the other players. Let's see what kind of schedule we can come up with:

In [4]:
%time report(hillclimb(P=10))

Round 1: (6, 7) vs (0, 5); (3, 4) vs (2, 8)
Round 2: (1, 8) vs (0, 3); (7, 9) vs (4, 5)
Round 3: (3, 6) vs (1, 7); (0, 9) vs (2, 5)
Round 4: (2, 9) vs (6, 8); (1, 3) vs (4, 7)
Round 5: (0, 8) vs (5, 7); (4, 6) vs (2, 3)
Round 6: (2, 4) vs (3, 5); (1, 6) vs (8, 9)
Round 7: (6, 9) vs (3, 7); (1, 2) vs (5, 8)
Round 8: (1, 4) vs (5, 9); (0, 7) vs (3, 8)
Round 9: (1, 5) vs (2, 7); (3, 9) vs (0, 6)
Round 10: (7, 8) vs (4, 9); (0, 1) vs (2, 6)
Round 11: (4, 8) vs (5, 6); (0, 2) vs (1, 9)

22 games in 11 rounds for 10 players
Number of times each player plays against each opponent:

  | 0 1 2 3 4 5 6 7 8 9  Total
--+--------------------  -----
 0| - 2 2 2 - 2 2 2 2 2   8
 1| 2 - 3 2 1 2 2 2 2 2   9
 2| 2 3 - 2 2 3 2 - 2 2   9
 3| 2 2 2 - 3 - 3 3 2 1   9
 4| - 1 2 3 - 3 1 2 2 2   8
 5| 2 2 3 - 3 - 1 3 2 2   9
 6| 2 2 2 3 1 1 - 2 2 3   9
 7| 2 2 - 3 2 3 2 - 2 2   9
 8| 2 2 2 2 2 2 2 2 - 2   9
 9| 2 2 2 1 2 2 3 2 2 -   9
CPU times: user 2min 39s, sys: 661 ms, total: 2min 40s
Wall time: 2min 43s


In this schedule several players never play each other; it may be possible to improve on that (in another run that has better luck with random numbers).

# 16 Player Tournament

Let's jump to 16 players on 4 courts (this will take a while):

In [5]:
%time report(hillclimb(P=16, C=4))

Round 1: (0, 12) vs (9, 13); (5, 10) vs (11, 15); (6, 8) vs (1, 3); (2, 7) vs (4, 14)
Round 2: (5, 12) vs (0, 10); (6, 11) vs (3, 9); (8, 15) vs (2, 14)
Round 3: (12, 15) vs (4, 6); (10, 13) vs (1, 9); (2, 5) vs (8, 11)
Round 4: (11, 14) vs (0, 9); (3, 13) vs (7, 10); (2, 15) vs (4, 12)
Round 5: (10, 11) vs (0, 15); (12, 14) vs (5, 13); (1, 8) vs (6, 9); (3, 7) vs (2, 4)
Round 6: (3, 11) vs (8, 13); (7, 9) vs (5, 15); (1, 6) vs (4, 10); (2, 12) vs (0, 14)
Round 7: (3, 10) vs (7, 12); (1, 14) vs (5, 11); (6, 13) vs (4, 8)
Round 8: (4, 5) vs (0, 8); (6, 10) vs (2, 11); (1, 13) vs (9, 15)
Round 9: (3, 5) vs (2, 9); (10, 15) vs (1, 7); (0, 11) vs (6, 12); (8, 14) vs (4, 13)
Round 10: (1, 10) vs (3, 8); (6, 7) vs (5, 9); (11, 12) vs (4, 15)
Round 11: (4, 7) vs (1, 11); (9, 14) vs (10, 12); (0, 6) vs (2, 13)
Round 12: (10, 14) vs (5, 8); (9, 12) vs (2, 3); (4, 11) vs (7, 13)
Round 13: (7, 8) vs (0, 13); (3, 12) vs (1, 5); (14, 15) vs (4, 9)
Round 14: (0, 5) vs (1, 4); (13, 14) vs (3, 15); (9

We get a pretty good schedule, although it takes 19 rounds rather than the 15 it would take if every court was filled, and again there are some players who never face each other.